In [11]:
# get an AES cipher
import chipwhisperer as cw
from chipwhisperer.common.utils.aes_cipher import AESCipher, aes_tables
import chipwhisperer.analyzer as cwa
ktp = cw.ktp.Basic()
key = list(ktp.next()[0])
for i in range(10):
    key.extend(cwa.aes_funcs.key_schedule_rounds(key[0:16], 0, i+1))
    
cipher = AESCipher(key)
print(bytearray(key))

CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c a0 fa fe 17 88 54 2c b1 23 a3 39 39 2a 6c 76 05 f2 c2 95 f2 7a 96 b9 43 59 35 80 7a 73 59 f6 7f 3d 80 47 7d 47 16 fe 3e 1e 23 7e 44 6d 7a 88 3b ef 44 a5 41 a8 52 5b 7f b6 71 25 3b db 0b ad 00 d4 d1 c6 f8 7c 83 9d 87 ca f2 b8 bc 11 f9 15 bc 6d 88 a3 7a 11 0b 3e fd db f9 86 41 ca 00 93 fd 4e 54 f7 0e 5f 5f c9 f3 84 a6 4f b2 4e a6 dc 4f ea d2 73 21 b5 8d ba d2 31 2b f5 60 7f 8d 29 2f ac 77 66 f3 19 fa dc 21 28 d1 29 41 57 5c 00 6e d0 14 f9 a8 c9 ee 25 89 e1 3f 0c c8 b6 63 0c a6')


In [87]:
import random

def generate_glitch(pt, cipher):
    # Do AES, but stop before the last Mix Columns
    state = list(pt)
    state = state+[16-len(state)]*(16-len(state))
    cipher._add_round_key(state, 0)
    
    # we break out after one round
    cipher._sub_bytes(state)
    cipher._shift_rows(state)
    print(state)
    cipher._mix_columns(state, False)
    print(state)
    cipher._add_round_key(state, 1)
    
    
    #last round
    cipher._sub_bytes(state)
    cipher._shift_rows(state)
    cipher._add_round_key(state, 10)
    return state

In [95]:
cts = []
for i in range(256):
    e = [i]*16
    #e.extend([4]*15)
    cts.append(generate_glitch(e, cipher))

[241, 228, 89, 235, 52, 104, 132, 71, 98, 138, 89, 36, 1, 243, 181, 196]
[124, 34, 129, 120, 19, 52, 134, 62, 60, 162, 54, 61, 125, 252, 212, 214]
[229, 121, 250, 39, 165, 66, 47, 240, 172, 139, 250, 92, 48, 210, 102, 167]
[135, 37, 26, 249, 72, 160, 178, 98, 99, 232, 44, 38, 204, 130, 220, 177]
[165, 145, 240, 178, 229, 230, 227, 250, 211, 189, 240, 73, 43, 16, 112, 126]
[187, 37, 2, 234, 249, 246, 203, 222, 216, 240, 78, 177, 104, 229, 89, 225]
[52, 149, 71, 117, 241, 191, 41, 89, 194, 75, 71, 6, 103, 255, 62, 61]
[254, 185, 176, 100, 83, 182, 247, 44, 3, 155, 13, 93, 215, 253, 163, 18]
[21, 172, 130, 7, 113, 13, 179, 201, 121, 31, 130, 58, 215, 218, 246, 100]
[64, 204, 175, 31, 143, 108, 65, 164, 107, 224, 55, 98, 82, 29, 86, 134]
[49, 98, 202, 18, 216, 137, 214, 125, 228, 116, 202, 10, 254, 33, 14, 93]
[28, 162, 234, 223, 128, 205, 97, 214, 143, 67, 1, 157, 215, 243, 36, 140]
[216, 194, 125, 128, 49, 161, 59, 202, 149, 221, 125, 224, 118, 188, 72, 25]
[11, 64, 123, 215, 107, 239, 1

In [96]:
def decrypt_last_round(ct, key_guess):
    state = ct ^ key_guess
    state = aes_tables.i_sbox[state]
    return state

In [97]:
print(cts)

[[86, 196, 143, 206, 221, 146, 31, 33, 33, 95, 222, 187, 237, 2, 160, 84], [28, 171, 160, 37, 115, 93, 137, 161, 232, 23, 101, 174, 56, 253, 7, 102], [127, 46, 12, 193, 106, 3, 48, 221, 238, 152, 188, 96, 154, 253, 152, 98], [136, 140, 225, 88, 112, 233, 38, 6, 86, 190, 35, 150, 226, 121, 181, 229], [49, 19, 82, 68, 12, 244, 146, 185, 179, 156, 221, 145, 10, 102, 48, 159], [181, 250, 254, 15, 249, 15, 37, 97, 112, 228, 246, 77, 226, 9, 239, 239], [178, 254, 30, 99, 216, 98, 224, 51, 236, 83, 155, 160, 228, 151, 127, 40], [21, 17, 34, 79, 157, 211, 100, 10, 196, 139, 151, 76, 64, 65, 104, 238], [59, 78, 120, 0, 144, 54, 85, 52, 22, 31, 27, 53, 213, 66, 5, 54], [95, 147, 57, 23, 246, 159, 123, 166, 124, 172, 97, 198, 250, 34, 155, 51], [31, 151, 242, 228, 35, 16, 96, 77, 117, 121, 17, 167, 205, 124, 200, 162], [74, 217, 110, 192, 46, 82, 16, 144, 3, 152, 126, 238, 132, 128, 45, 234], [170, 152, 9, 145, 42, 171, 97, 39, 96, 41, 165, 250, 69, 158, 138, 229], [45, 147, 103, 217, 190, 244, 6

In [110]:

guessed_key = []
for kbyte in range(16):
    probable_guesses = []
    for key_guess in range(256):
        c = 0
        for ct in cts:
            c ^= decrypt_last_round(ct[kbyte], key_guess)
        if c == 0:
            probable_guesses.append(key_guess)
    guessed_key.append(probable_guesses)

In [111]:
probable_guesses

[166]

In [112]:
print(guessed_key)

[[81, 95, 208], [20], [43, 214, 249], [13, 22, 168, 217], [201, 204], [238], [37, 191, 241], [137, 241], [225], [63], [12, 211, 215], [200], [96, 182, 234], [99], [12, 69, 127], [166]]


In [203]:
# get an AES cipher
import chipwhisperer as cw
from chipwhisperer.common.utils.aes_cipher import AESCipher, aes_tables
import chipwhisperer.analyzer as cwa
ktp = cw.ktp.Basic()
key = list(ktp.next()[0])
for i in range(10):
    key.extend(cwa.aes_funcs.key_schedule_rounds(key[0:16], 0, i+1))
    
cipher = AESCipher(key)
print(bytearray(key))

import random

def generate_glitch(pt, cipher):
    # Do AES, but stop before the last Mix Columns
    state = list(pt)
    state = state+[16-len(state)]*(16-len(state))
    cipher._add_round_key(state, 0)
    
    # we break out after one round
    cipher._sub_bytes(state)
    cipher._shift_rows(state)
    cipher._mix_columns(state, False)
    cipher._add_round_key(state, 1)
    
    #last round
    cipher._sub_bytes(state)
    cipher._shift_rows(state)
    x = list(state)
    cipher._add_round_key(state, 2)
    cipher._add_round_key(x, 0)
    return x, state

CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c a0 fa fe 17 88 54 2c b1 23 a3 39 39 2a 6c 76 05 f2 c2 95 f2 7a 96 b9 43 59 35 80 7a 73 59 f6 7f 3d 80 47 7d 47 16 fe 3e 1e 23 7e 44 6d 7a 88 3b ef 44 a5 41 a8 52 5b 7f b6 71 25 3b db 0b ad 00 d4 d1 c6 f8 7c 83 9d 87 ca f2 b8 bc 11 f9 15 bc 6d 88 a3 7a 11 0b 3e fd db f9 86 41 ca 00 93 fd 4e 54 f7 0e 5f 5f c9 f3 84 a6 4f b2 4e a6 dc 4f ea d2 73 21 b5 8d ba d2 31 2b f5 60 7f 8d 29 2f ac 77 66 f3 19 fa dc 21 28 d1 29 41 57 5c 00 6e d0 14 f9 a8 c9 ee 25 89 e1 3f 0c c8 b6 63 0c a6')


In [ ]:
from tqdm.notebook import trange
def gmul(a, b): 
    p = 0
    while a and b:
        if b & 1:
            p ^= a
        if (a & 0x80):
            a = (a << 1) ^ 0x11b;
        else:
            a <<= 1
        b >>= 1
    return p

for a in trange(255):
    for b in range(255):
        for c in range(255):
            for d in range(255):
                

In [207]:
print(0x2b)

43


In [222]:
print(len(solns))

251


In [231]:
cts = []
n_wrong = 0
for i in range(6,7):
    e = [i]*16
    print(e)
    #e.extend([4]*15)
    ct, ct2 = generate_glitch(e, cipher)
    for i in range(4):
        for j in range(4):
            print("Known[{}]={};".format(48-(i*4+j), ct[i+j*4]))
    #print(ct)
    #print(ct2)
    ct3 = [ct[i]^ct2[i] for i in range(16)]

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
Known[48]=73;
Known[47]=57;
Known[46]=166;
Known[45]=91;
Known[44]=148;
Known[43]=34;
Known[42]=155;
Known[41]=59;
Known[40]=242;
Known[39]=23;
Known[38]=130;
Known[37]=60;
Known[36]=221;
Known[35]=28;
Known[34]=224;
Known[33]=178;


In [210]:
73 ^ 238

167

In [196]:
a1, a2 = generate_glitch(ct, cipher)

In [198]:
bytearray(a2)

CWbytearray(b'93 f4 f3 d8 a6 c8 b3 e0 7d 27 b2 0b f9 49 16 fe')

In [199]:
a1

[74, 72, 115, 60, 244, 240, 216, 5, 143, 229, 39, 249, 131, 223, 175, 189]

In [174]:
print(key[32:])

[242, 194, 149, 242, 122, 150, 185, 67, 89, 53, 128, 122, 115, 89, 246, 127, 61, 128, 71, 125, 71, 22, 254, 62, 30, 35, 126, 68, 109, 122, 136, 59, 239, 68, 165, 65, 168, 82, 91, 127, 182, 113, 37, 59, 219, 11, 173, 0, 212, 209, 198, 248, 124, 131, 157, 135, 202, 242, 184, 188, 17, 249, 21, 188, 109, 136, 163, 122, 17, 11, 62, 253, 219, 249, 134, 65, 202, 0, 147, 253, 78, 84, 247, 14, 95, 95, 201, 243, 132, 166, 79, 178, 78, 166, 220, 79, 234, 210, 115, 33, 181, 141, 186, 210, 49, 43, 245, 96, 127, 141, 41, 47, 172, 119, 102, 243, 25, 250, 220, 33, 40, 209, 41, 65, 87, 92, 0, 110, 208, 20, 249, 168, 201, 238, 37, 137, 225, 63, 12, 200, 182, 99, 12, 166]


In [165]:
hex(decrypt_last_round(0xee, 242))

'0xc4'

In [167]:
hex(decrypt_last_round(0x7b, 194))

'0xdb'

In [168]:
print(hex(160))

0xa0
